In [49]:
import time
import re
import datetime
import hashlib
import pytz
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

options = Options()
options.add_argument('--headless') # ヘッドレスモードを有効にする

driver = webdriver.Chrome(chrome_options=options)
driver.set_page_load_timeout(10) #ページロードのタイムアウト(秒)

driver_wait = WebDriverWait(driver, 10) #操作した時のタイムアウト時間

# AT-Xの番組紹介ページから放送日を取得する

driver.get('https://www.at-x.com/encount/')
driver_wait.until(ec.presence_of_all_elements_located) #処理待ち

assert '東京エンカウント' in driver.title

print(driver.current_url)
data = driver.page_source.encode('utf-8')
soup = BeautifulSoup(data, 'lxml')
print(soup.title.string)

# ---------------

schedule = soup.find(id="schedule")

ttl_lv03 = schedule.find_all(class_='ttl_lv03')
time_table = schedule.find_all(class_='time_table')

def get_year_from_tag(tag):
    m = re.search('(\d{4})年', tag.text)
    assert m != None
    return int(m.group(1))
def get_day_from_tag(tag):
    m = re.search('(\d{1,2})月\D*(\d{1,2})日\D*(\d{1,2})\：(\d{1,2})', tag.text)
    assert m != None
    mo = int(m.group(1))
    day = int(m.group(2))
    h = int(m.group(3))
    m = int(m.group(4))
    return (mo, day, h, m)
def create_datetime(yyyy, mo, day, h, m, timezone):
    dt = datetime.datetime(yyyy, mo, day, h, m)
    tz = pytz.timezone(timezone)
    date = tz.localize(dt)
    return date

schedule_list = []
for h3, div in zip(ttl_lv03, time_table):
    story_number = h3.find(class_='icon_number')
    year = div.find_all(class_='year')[0]
    days = div.find_all(class_='days')[0].find_all('li')
    for day in days:
        yyyy = get_year_from_tag(year)
        mo, day, h, m = get_day_from_tag(day)
        date = create_datetime(yyyy, mo, day, h, m, 'Asia/Tokyo')
        schedule_list.append((len(schedule_list), date.timestamp(), story_number.text))

schedule_list.sort(key=lambda x:x[1]) #放送日順
print(schedule_list)

# ---------------

new_story_title = soup.find(id='right_contents').find(class_='ttl_lv02').text.strip()
new_story_body = soup.find(id='right_contents').find(class_='contents').text.strip()

print(new_story_title)
print(new_story_body)

# ---------------

schedule_digest = hashlib.sha224(str(schedule.text).encode('utf-8')).hexdigest()
print('schedule_digest : ', schedule_digest)


# # 検索語を入力して送信する。
# input_element = driver.find_element_by_name('q')
# input_element.send_keys('Python')
# input_element.send_keys(Keys.RETURN)

# time.sleep(2)  # Chromeの場合はAjaxで遷移するので、とりあえず適当に2秒待つ。

# # タイトルに'Python'が含まれていることを確認する。
# assert 'Python' in driver.title

# # スクリーンショットを撮る。
# driver.save_screenshot('search_results.png')

# # 検索結果を表示する。
# for a in driver.find_elements_by_css_selector('h3 > a'):
#     print(a.text)
#     print(a.get_attribute('href'))

driver.quit()  # ブラウザーを終了する。

https://www.at-x.com/encount/
東京エンカウント弐 | 番組 | AT-X ワンランク上のアニメ専門チャンネル
[(0, 1517664600.0, '第55章'), (1, 1517895000.0, '第55章'), (2, 1518125400.0, '第55章'), (3, 1518269400.0, '第55章'), (4, 1518499800.0, '第55章'), (5, 1518730200.0, '第55章'), (6, 1518874200.0, '第56章'), (7, 1519104600.0, '第56章'), (8, 1519335000.0, '第56章'), (9, 1519479000.0, '第56章'), (10, 1519709400.0, '第56章'), (11, 1519939800.0, '第56章')]
第五十五章の予告
今回も新たなゲストがやってきます！

なんと今回のゲストは...

声優の

細谷佳正さんが登場！

細谷さんのリクエストにお答えして
３人であのゲームをマルチプレイします。

お楽しみに！
schedule_digest :  4c6d3ade0f878f872e04413ffa34c68335132c898fe59344ba2400b9


In [46]:
import re
import datetime
import pytz
import hashlib

year = '\n        2018年'
print(type(year))

m = re.search('(\d{4})年', year)
yyyy = int(m.group(1))

day = '\n      2月 3日（土）22：30'
m = re.search('(\d{1,2})月\D*(\d{1,2})日\D*(\d{1,2})\：(\d{1,2})', day)
mo = int(m.group(1))
day = int(m.group(2))
h = int(m.group(3))
m = int(m.group(4))

# date = datetime.datetime(yyyy, mo, day, h, m, tzinfo=pytz.timezone('Asia/Tokyo'))
dt = datetime.datetime(yyyy, mo, day, h, m)
tz = pytz.timezone('Asia/Tokyo')
date = tz.localize(dt)
print(date, date.timestamp())

"""
(2018, 2, 3, 22, 30), 1517663460.0

GMT: 2018年2月3日 13:30:00 Saturday
Your time zone: 2018年2月3日 22:30:00 土曜日 GMT+09:00
"""

story = '今回も新たなゲストがやってきます！'
schedule_digest = hashlib.sha224(str(story).encode('utf-8')).hexdigest()
print('schedule_digest : ', schedule_digest)

<class 'str'>
2018-02-03 22:30:00+09:00 1517664600.0
schedule_digest :  ca7fc3dde39ea1400949ec7a507ab40af2748d393732c7b6375bc585
